In [1]:
# !wget https://zenodo.org/record/3951095/files/simulated_cyp2d6_diplotypes.tar.gz -O simulated.tgz

In [2]:
# !tar -xsf simulated.tgz

In [3]:
# !ls -1 ./simulated_cyp2d6_diplotypes/ | wc -l

In [4]:
import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np

In [5]:
!git clone https://github.com/Locrian24/seng474-term-project.git

fatal: destination path 'seng474-term-project' already exists and is not an empty directory.


In [6]:
!cd seng474-term-project/ && git pull

Already up to date.


In [7]:
device_name = tf.test.gpu_device_name()
device_name

'/device:GPU:0'

In [8]:
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TF_MASTER)
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.TPUStrategy(resolver)

In [9]:
import sys
sys.path.insert(0, '/content/seng474-term-project/step_1')

In [10]:
from encode_first_step import FirstStep2Seq

In [11]:
# Global variables rn for testing

ANNOTATIONS = '/content/seng474-term-project/step_1/data/gvcf2seq.annotation_embeddings.csv'
EMBEDDINGS = '/content/seng474-term-project/step_1/data/embeddings.txt'
REF = '/content/seng474-term-project/step_1/data/ref.seq'

In [12]:
# encoding = FirstStep2Seq(vcf=VCF, labels=LABEL, annotation_file=ANNOTATIONS, embedding_file=EMBEDDINGS, ref_seq=REF)

In [13]:
# encoding.X.shape

In [14]:
print(tf.__version__)
import distutils
if distutils.version.LooseVersion(tf.__version__) < '1.14':
    raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/classification_iris_data_with_keras.ipynb')

2.5.0


In [15]:
import pathlib
import tensorflow as tf

def get_batch_files(training_count, test_count):

  file_root = tf.keras.utils.get_file(
      'simulated_cyp2d6_diplotypes',
      'https://zenodo.org/record/3951095/files/simulated_cyp2d6_diplotypes.tar.gz',
      untar=True
  )
  file_root = pathlib.Path(file_root)
  filenames = []
  for f in file_root.glob("*"):
    filenames.append(f)

  _filenames = np.array([f.name.split('.')[0] for f in filenames])
  batch_names = np.unique(_filenames)
  filenames = np.array([str(f.absolute()) for f in filenames])
  training_batches, test_batches = [], []

  for i, b in enumerate(batch_names):
    if i >= test_count + training_count:
      break
      
    if i < training_count:
      training_batches.append(filenames[_filenames == b])
    else:
      test_batches.append(filenames[_filenames == b])

  return training_batches, test_batches

In [16]:
from tensorflow.keras.utils import to_categorical

def hot_encode_float(y):
  classes = []
  values = np.unique(y)
  for i in range(len(values)):
    classes.append(str(i))
  encoded_classes = to_categorical(classes)
  conversion_dict = dict(zip(values, range(5)))
  encoded_y = np.array([encoded_classes[conversion_dict[i]] for i in y])

  return encoded_y

def generate_data(batches):
  for filenames in batches:
    vcf = 0 if 'vcf' == filenames[0].decode('utf-8').split('.')[-1] else 1
    labels = 1 - vcf
    encoding = FirstStep2Seq(vcf=filenames[vcf].decode('utf-8'), labels=filenames[labels].decode('utf-8'), embedding_file=EMBEDDINGS, annotation_file=ANNOTATIONS, ref_seq=REF)
    y = hot_encode_float(encoding.y.flatten())
    for i in range(encoding.X.shape[0]):
      yield encoding.X[i], y[i]

In [17]:
# First train on one file to make sure things work
from tensorflow import keras

In [18]:
def get_model():
  return keras.Sequential([
    keras.layers.Conv1D(70, kernel_size=19, strides=5,input_shape=(14868, 13), batch_input_shape=(None, 14868, 13), activation=keras.activations.linear, kernel_initializer=keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name = "conv1d_1"),
    keras.layers.BatchNormalization(name="batch_1"),
    keras.layers.ReLU(name="relu_1"),
    keras.layers.MaxPooling1D(pool_size=3, strides=3, name="maxpooling_1"),
    keras.layers.Conv1D(46, kernel_size=11, strides=5, activation=keras.activations.linear, kernel_initializer=keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name = "conv1d_2"),
    keras.layers.BatchNormalization(name="batch_2"),
    keras.layers.ReLU(name="relu_2"),
    keras.layers.MaxPooling1D(pool_size=4, strides=4, name="maxpooling_2"),
    keras.layers.Conv1D(46, kernel_size=7, strides=5, activation=keras.activations.linear, kernel_initializer=keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name = "conv1d_3"),
    keras.layers.BatchNormalization(name="batch_3"),
    keras.layers.ReLU(name="relu_3"),
    keras.layers.MaxPooling1D(pool_size=4, strides=4, name="maxpooling_3"),
    keras.layers.Flatten(name="flatten_3"),
    keras.layers.Dense(32, activation=keras.activations.relu, kernel_initializer=keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name="dense_4"),
    keras.layers.Dropout(rate=0.03, name="dropout_4"),
    keras.layers.Dense(5, activation='softmax', kernel_initializer=keras.initializers.VarianceScaling(mode='fan_avg', distribution='uniform'), name="dense_5"),
  ])

In [19]:
training_batches, test_batches = get_batch_files(100, 20)

In [20]:
train_dataset = tf.data.Dataset.from_generator(generate_data, args=[training_batches], output_types=(tf.float32, tf.float32), output_shapes=((14868, 13), (5,)))

In [21]:
batch_size = 100
steps_per_epoch = 50000 // batch_size

In [22]:
batched = train_dataset.shuffle(500).repeat(count=2).batch(batch_size)

In [23]:
with tf.device('/device:GPU:0'):
  model = get_model()
  adam = keras.optimizers.Adam(learning_rate=0.001)
  model.compile(optimizer=adam,
                loss=tf.keras.losses.CategoricalCrossentropy(), 
                metrics=['accuracy'])
  model.load_weights('weights.h5')
  # model.fit(batched, epochs=2, steps_per_epoch=steps_per_epoch)

In [24]:
test_dataset = tf.data.Dataset.from_generator(generate_data, args=[test_batches], output_types=(tf.float32, tf.float32), output_shapes=((14868, 13), (5,)))

In [25]:
sample = tf.data.experimental.sample_from_datasets([test_dataset])

In [26]:
# model.save_weights('weights.h5')

In [44]:
for filenames in test_batches:
  vcf = 0 if 'vcf' == filenames[0].split('.')[-1] else 1
  labels = 1 - vcf
  encoding = FirstStep2Seq(vcf=filenames[vcf], labels=filenames[labels], embedding_file=EMBEDDINGS, annotation_file=ANNOTATIONS, ref_seq=REF)
  y = hot_encode_float(encoding.y.flatten())
  print(model.evaluate(encoding.X, y))

16/16 [==============================] - 0s 18ms/step - loss: 0.6115 - accuracy: 0.7800
[0.6114830374717712, 0.7799999713897705]
16/16 [==============================] - 0s 18ms/step - loss: 0.6545 - accuracy: 0.7740
[0.6544732451438904, 0.7739999890327454]
16/16 [==============================] - 0s 19ms/step - loss: 0.5879 - accuracy: 0.7840
[0.5878723859786987, 0.7839999794960022]
16/16 [==============================] - 0s 21ms/step - loss: 0.6648 - accuracy: 0.7700
[0.6647749543190002, 0.7699999809265137]
16/16 [==============================] - 0s 20ms/step - loss: 0.6741 - accuracy: 0.7720
[0.674068808555603, 0.7720000147819519]
16/16 [==============================] - 0s 21ms/step - loss: 0.6235 - accuracy: 0.7700
[0.623478353023529, 0.7699999809265137]
16/16 [==============================] - 0s 19ms/step - loss: 0.6597 - accuracy: 0.7780
[0.6597424149513245, 0.777999997138977]
16/16 [==============================] - 0s 20ms/step - loss: 0.5749 - accuracy: 0.8180
[0.574947595